# Vanilla PPO - with ActionWrapper

Run on 3 environments

Empty Room

Empty Room Random

Four Rooms

In [21]:
import gymnasium as gym
import minigrid
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from minigrid.wrappers import FlatObsWrapper
import time
import numpy as np
from minigrid.core.world_object import Goal
import random
from stable_baselines3.common.callbacks import BaseCallback
from gymnasium.envs.registration import register

# First Empty

In [2]:
# Create and wrap the environment
env_id = "MiniGrid-Empty-16x16-v0"

In [3]:
class ActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super(ActionWrapper, self).__init__(env)
        # Define a new action space with only the relevant actions
        self.action_space = gym.spaces.Discrete(3)  # Only three actions: left, right, forward

    def action(self, action):
        # Map the new actions to the original actions
        action_mapping = {
            0: 0,  # left
            1: 1,  # right
            2: 2   # forward
        }
        return action_mapping[action]

In [4]:
def create_env():
    env = gym.make(env_id, render_mode="rgb_array")#,max_episode_steps=200)
    env = ActionWrapper(env)
    env = FlatObsWrapper(env)  
    return env

# Use the function in make_vec_env
env_vec = make_vec_env(create_env, n_envs=1)
env_vec2 = make_vec_env(create_env, n_envs=4)

In [5]:
model = PPO("MlpPolicy", env_vec2, verbose=1,
            tensorboard_log="log/ppo_action_empty_no_step_cap"
            )

# Train the model
model.learn(total_timesteps=200000)

Using cpu device
Logging to log/ppo_action_empty_no_step_cap/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.02e+03 |
|    ep_rew_mean     | 0.0133   |
| time/              |          |
|    fps             | 5386     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 711         |
|    ep_rew_mean          | 0.337       |
| time/                   |             |
|    fps                  | 2780        |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.023368314 |
|    clip_fraction        | 0.212       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.08       |
|    

In [6]:
model.save("models/ppo_action_empty_no_step_cap")

In [7]:
model = PPO.load("models/ppo_action_empty_no_step_cap",env=env_vec)

mean_reward, std_reward = evaluate_policy(model, env=env_vec, n_eval_episodes=50)
print(f"Mean Reward: {mean_reward} +/- {std_reward}")

Mean Reward: 0.975391 +/- 0.0


In [8]:
num_episodes = 50
total_rewards = []  # List to store total rewards for each episode

for episode in range(num_episodes):
    obs = env_vec.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env_vec.step(action)
        #print(done)
        total_reward += reward
        env_vec.render('human')  # Render the environment at each step
        #time.sleep(0.05)  # Adjust this to control the speed of the rendering
    total_rewards.append(total_reward)  # Store the total reward for this episode
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

# Calculate the average reward across all episodes
average_reward = sum(total_rewards) / num_episodes
print(f"Average Reward over {num_episodes} episodes: {average_reward}")

env_vec.close()  # Close the environment when done

Episode 1: Total Reward = [0.9753906]
Episode 2: Total Reward = [0.9753906]
Episode 3: Total Reward = [0.9753906]
Episode 4: Total Reward = [0.9753906]
Episode 5: Total Reward = [0.9753906]
Episode 6: Total Reward = [0.9753906]
Episode 7: Total Reward = [0.9753906]
Episode 8: Total Reward = [0.9736328]
Episode 9: Total Reward = [0.9753906]
Episode 10: Total Reward = [0.9753906]
Episode 11: Total Reward = [0.9753906]
Episode 12: Total Reward = [0.9753906]
Episode 13: Total Reward = [0.9753906]
Episode 14: Total Reward = [0.9753906]
Episode 15: Total Reward = [0.9753906]
Episode 16: Total Reward = [0.9753906]
Episode 17: Total Reward = [0.9753906]
Episode 18: Total Reward = [0.9753906]
Episode 19: Total Reward = [0.9753906]
Episode 20: Total Reward = [0.9753906]
Episode 21: Total Reward = [0.9753906]
Episode 22: Total Reward = [0.9753906]
Episode 23: Total Reward = [0.9753906]
Episode 24: Total Reward = [0.9753906]
Episode 25: Total Reward = [0.9753906]
Episode 26: Total Reward = [0.9753

# Empty random room

In [22]:
register(
    id='custom_empty-v0',
    entry_point='emptyrandom:RandomGoalEmptyEnv',
    kwargs={}
)

/Users/anthony/.pyenv/versions/3.10.13/envs/cs5756-project/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment custom_empty-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [23]:
#gym.pprint_registry()  # to see all registered environments

if 'custom_empty-v0' in gym.envs.registry:
    print("Environment 'custom_empty-v0' is registered.")
else:
    print("Environment 'custom_empty-v0' is NOT registered.")

Environment 'custom_empty-v0' is registered.


In [24]:
env_id2 = 'custom_empty-v0'
def create_env():
    env = gym.make(env_id2, render_mode="rgb_array",size=16)
    env = ActionWrapper(env)
    env = FlatObsWrapper(env)  
    return env

env_vec_ran = make_vec_env(create_env, n_envs=1)
env_vec_ran2 = make_vec_env(create_env, n_envs=4)

In [26]:
# Initialize the PPO model
model2 = PPO("MlpPolicy", env_vec_ran2, verbose=1,
            tensorboard_log="log/ppo_action_empty_random_no_step_cap"
            )

# Train the model
model2.learn(total_timesteps=200000)

Using cpu device
Logging to log/ppo_action_empty_random_no_step_cap/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 428      |
|    ep_rew_mean     | 0.611    |
| time/              |          |
|    fps             | 5463     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 8192     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 531         |
|    ep_rew_mean          | 0.505       |
| time/                   |             |
|    fps                  | 3056        |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.015681185 |
|    clip_fraction        | 0.222       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       

In [13]:
model2.save("models/ppo_action_empty_random_no_step_cap")

In [14]:
model2 = PPO.load("models/ppo_action_empty_random_no_step_cap",env=env_vec_ran)

mean_reward, std_reward = evaluate_policy(model2, env=env_vec_ran, n_eval_episodes=50)
print(f"Mean Reward: {mean_reward} +/- {std_reward}")

Mean Reward: 0.90226366 +/- 0.26621997143201787


In [15]:
num_episodes = 50
total_rewards = []  # List to store total rewards for each episode

for episode in range(num_episodes):
    obs = env_vec_ran.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model2.predict(obs)
        obs, reward, done, info = env_vec_ran.step(action)
        #print(done)
        total_reward += reward
        env_vec_ran.render('human')  # Render the environment at each step
        #time.sleep(0.05)  # Adjust this to control the speed of the rendering
    total_rewards.append(total_reward)  # Store the total reward for this episode
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

# Calculate the average reward across all episodes
average_reward = sum(total_rewards) / num_episodes
print(f"Average Reward over {num_episodes} episodes: {average_reward}")

env_vec_ran.close()  # Close the environment when done

Episode 1: Total Reward = [0.99208987]
Episode 2: Total Reward = [0.95341796]
Episode 3: Total Reward = [0.9964844]
Episode 4: Total Reward = [0.95166016]
Episode 5: Total Reward = [0.99560547]
Episode 6: Total Reward = [0.9771484]
Episode 7: Total Reward = [0.98154294]
Episode 8: Total Reward = [0.99560547]
Episode 9: Total Reward = [0.99296874]
Episode 10: Total Reward = [0.95341796]
Episode 11: Total Reward = [0.990332]
Episode 12: Total Reward = [0.9824219]
Episode 13: Total Reward = [0.9806641]
Episode 14: Total Reward = [0.9736328]
Episode 15: Total Reward = [0.990332]
Episode 16: Total Reward = [0.97978514]
Episode 17: Total Reward = [0.95341796]
Episode 18: Total Reward = [0.9701172]
Episode 19: Total Reward = [0.97978514]
Episode 20: Total Reward = [0.9692383]
Episode 21: Total Reward = [0.99472654]
Episode 22: Total Reward = [0.98154294]
Episode 23: Total Reward = [0.9850586]
Episode 24: Total Reward = [0.9736328]
Episode 25: Total Reward = [0.94990236]
Episode 26: Total Rewa

# Now Four rooms

In [16]:
env_id3 = "MiniGrid-FourRooms-v0"
def create_env():
    env = gym.make(env_id3, render_mode="rgb_array")
    env = ActionWrapper(env)
    env = FlatObsWrapper(env)  
    return env

env_vec_four = make_vec_env(create_env, n_envs=1)
env_vec_four2 = make_vec_env(create_env, n_envs=4)

In [17]:
# Initialize the PPO model
model3 = PPO("MlpPolicy", env_vec_four2, verbose=1,
            tensorboard_log="log/ppo_action_four_no_step_cap"
            )

# Train the model
model3.learn(total_timesteps=1000000)

Using cpu device
Logging to log/ppo_action_four_no_step_cap/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 98.8     |
|    ep_rew_mean     | 0.0129   |
| time/              |          |
|    fps             | 4752     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 8192     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 96.4       |
|    ep_rew_mean          | 0.0376     |
| time/                   |            |
|    fps                  | 2769       |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 16384      |
| train/                  |            |
|    approx_kl            | 0.01376416 |
|    clip_fraction        | 0.169      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.09      |
|    explained_varia

In [18]:
model3.save("models/ppo_action_four_no_step_cap")

In [19]:
model3 = PPO.load("models/ppo_action_four_no_step_cap",env=env_vec_four)

mean_reward, std_reward = evaluate_policy(model3, env=env_vec_four, n_eval_episodes=50)
print(f"Mean Reward: {mean_reward} +/- {std_reward}")

Mean Reward: 0.22526000000000004 +/- 0.3828030726104481


In [20]:
num_episodes = 50
total_rewards = []  # List to store total rewards for each episode

for episode in range(num_episodes):
    obs = env_vec_four.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model3.predict(obs)
        obs, reward, done, info = env_vec_four.step(action)
        #print(done)
        total_reward += reward
        env_vec_four.render('human')  # Render the environment at each step
        #time.sleep(0.05)  # Adjust this to control the speed of the rendering
    total_rewards.append(total_reward)  # Store the total reward for this episode
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

# Calculate the average reward across all episodes
average_reward = sum(total_rewards) / num_episodes
print(f"Average Reward over {num_episodes} episodes: {average_reward}")

env_vec_four.close()  # Close the environment when done

Episode 1: Total Reward = [0.667]
Episode 2: Total Reward = [0.]
Episode 3: Total Reward = [0.487]
Episode 4: Total Reward = [0.595]
Episode 5: Total Reward = [0.622]
Episode 6: Total Reward = [0.73]
Episode 7: Total Reward = [0.]
Episode 8: Total Reward = [0.]
Episode 9: Total Reward = [0.]
Episode 10: Total Reward = [0.]
Episode 11: Total Reward = [0.91]
Episode 12: Total Reward = [0.802]
Episode 13: Total Reward = [0.]
Episode 14: Total Reward = [0.]
Episode 15: Total Reward = [0.]
Episode 16: Total Reward = [0.46]
Episode 17: Total Reward = [0.]
Episode 18: Total Reward = [0.775]
Episode 19: Total Reward = [0.46]
Episode 20: Total Reward = [0.]
Episode 21: Total Reward = [0.802]
Episode 22: Total Reward = [0.]
Episode 23: Total Reward = [0.604]
Episode 24: Total Reward = [0.]
Episode 25: Total Reward = [0.964]
Episode 26: Total Reward = [0.]
Episode 27: Total Reward = [0.946]
Episode 28: Total Reward = [0.262]
Episode 29: Total Reward = [0.]
Episode 30: Total Reward = [0.91]
Episod